In [3]:
import pandas as pd
import joblib
import numpy as np

In [4]:
df1 = pd.read_csv('preprocessed.csv')
# area_coord_total = pd.read_csv('area_xy_total.csv')
area_coord_f = pd.read_csv('area_xy.csv')
# base_df = df1.merge(area_coord_f, left_on='itemID', right_on='VISIT_AREA_NM')

In [5]:
# exist_values = df1.itemID.unique()

In [34]:
# area_coord_total.assign(SIDO = lambda df: df.SIDO.apply(lambda x: '경기' if x not in ['경기', '서울', '인천'] else x)).query('VISIT_AREA_NM in @exist_values').to_csv('area_xy.csv', index = False, encoding = 'utf-8-sig')

In [6]:
df1

,userID,itemID,rating
0,e000004,화성 관광열차 안내소 연무대 매표소,3.666667
1,e000004,창룡문,4.000000
2,e000004,수원 화성 화홍문,3.333333
3,e000009,농협안성팜랜드,4.000000
4,e000009,스타필드 안성,5.000000
...,...,...,...
4108,g000018,AK플라자 수원,3.333333
4109,g000018,AK플라자 수원,3.000000
4110,g005600,국립과천과학관,5.000000
4111,g010556,방아머리 해변,3.333333


In [7]:
from surprise.dataset import DatasetAutoFolds, Reader, Dataset
from surprise import SVD 

In [148]:
test_id = 'e000004'
reader = Reader(line_format='user item rating timestamp', sep='\t', rating_scale= (1,5))
data = Dataset.load_from_df(df1[['userID', 'itemID', 'rating']], reader)
train_set = data.build_full_trainset()
rcsys = SVD(n_factors=10, n_epochs=20, random_state=9999)
rcsys.fit(train_set)
pred = [rcsys.predict(test_id, area_nm) for area_nm in df1.itemID.unique() if area_nm not in df1.query('userID==@test_id').itemID.values]
pred.sort(key = lambda x: x.est, reverse = True)


In [156]:
pd.DataFrame(pred)

,uid,iid,r_ui,est,details
0,e000004,KSPO DOME 올림픽 체조경기장,None,4.561581,{'was_impossible': False}
1,e000004,서울숲,None,4.526587,{'was_impossible': False}
2,e000004,KSPO 돔,None,4.495735,{'was_impossible': False}
3,e000004,석촌호수 산책길,None,4.469516,{'was_impossible': False}
4,e000004,국립중앙박물관,None,4.463946,{'was_impossible': False}
...,...,...,...,...,...
704,e000004,양수리 전통시장,None,3.702711,{'was_impossible': False}
705,e000004,말죽거리,None,3.679036,{'was_impossible': False}
706,e000004,SETEC,None,3.653733,{'was_impossible': False}
707,e000004,행궁동 벽화마을,None,3.636644,{'was_impossible': False}


In [157]:
pd.DataFrame(pred).head(5).merge(area_coord_f,how='left', left_on = 'iid', right_on = 'VISIT_AREA_NM')

,uid,iid,r_ui,est,details,VISIT_AREA_NM,X_COORD,Y_COORD,SIDO
0,e000004,KSPO DOME 올림픽 체조경기장,None,4.561581,{'was_impossible': False},KSPO DOME 올림픽 체조경기장,127.127408,37.519244,서울
1,e000004,서울숲,None,4.526587,{'was_impossible': False},서울숲,127.037618,37.544322,서울
2,e000004,KSPO 돔,None,4.495735,{'was_impossible': False},KSPO 돔,127.127400,37.519261,서울
3,e000004,석촌호수 산책길,None,4.469516,{'was_impossible': False},석촌호수 산책길,127.105721,37.510090,서울
4,e000004,국립중앙박물관,None,4.463946,{'was_impossible': False},국립중앙박물관,126.980203,37.523847,서울
